In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(-1, 28, 28, 1) / 255.0
x_test = x_test.reshape(-1, 28, 28, 1) / 255.0

def generate_random_number(number,x_data,y_data):
    num_samples = number
    data_numb=np.zeros((num_samples, 28, 100, 1), dtype = np.float32)
    data_labe=np.empty((num_samples, 1), dtype = np.chararray)
    for i in range(num_samples):
        digit1 = np.random.choice(y_data)
        digit2 = np.random.choice(y_data)
        digit3 = np.random.choice(y_data)
        digit4 = np.random.choice(y_data)
        digit5 = np.random.choice(y_data)
        
        length = np.random.randint(1, 6)
        canvas = np.zeros((28, 100, 1), dtype=np.float32)
        labe = np.zeros(length,dtype=np.int_)
        
        lent_front = 0
        for n in range(length):
            digit_labe =[digit1, digit2, digit3, digit4, digit5]
            digit = digit_labe[n]
            id=np.random.choice(np.where(y_train == digit)[0])
            if n==0:
                lent_front = 28*n
                x_data_front = np.zeros((28, lent_front, 1), dtype=np.float32)
                x_data_after = np.zeros((28, 72-lent_front, 1), dtype=np.float32)
            else:
                numb=np.random.randint(8,18)
                lent_front = lent_front+numb
                x_data_front = np.zeros((28, lent_front, 1), dtype=np.float32)
                x_data_after = np.zeros((28, 72-lent_front, 1), dtype=np.float32)
            canvas_add = np.concatenate([x_data_front, x_data[id]], 1)
            canvas_add = np.concatenate([canvas_add, x_data_after],  1)
            canvas += canvas_add
            labe[n] = y_data[id]
            
        datalabe = [str(x) for x in labe]
        data_labe[i] = ''.join(datalabe)
        data_numb[i]=canvas 
    print('end')
    return data_numb, data_labe

In [2]:

def get_label(y_lab):
    max_length = max(len(sublist[0]) for sublist in y_lab)
    lab_list = [[int(char) if len(char) == 1 else 11 for char in sublist[0]] + [10] * (max_length - len(sublist[0])) for sublist in y_lab]
    lab = np.array(lab_list)
    return lab

In [3]:

x_data_random , y_data_random = generate_random_number(100000,x_train, y_train)
y_data_ran_ctc = get_label(y_data_random)

end


In [4]:
from keras.utils import to_categorical
y_data_ran = to_categorical(y_data_ran_ctc, num_classes=11)

In [5]:
x_train_ran , y_train_ran, y_train_ran_ctc, y_train_random = x_data_random[:60000] , y_data_ran[:60000] , y_data_ran_ctc[:60000] , y_data_random[:60000]
x_test_ran , y_test_ran, y_test_ran_ctc, y_test_random= x_data_random[60000:80000] , y_data_ran[60000:80000] , y_data_ran_ctc[60000:80000] , y_data_random[60000:80000]
x_verify_ran , y_verify_ran, y_verify_ran_ctc, y_verify_random = x_data_random[80000:] , y_data_ran[80000:] , y_data_ran_ctc[80000:] , y_data_random[80000:]

In [40]:
from PIL import Image
from IPython.display import display

for id_n in np.random.randint(0, len(y_train_random),10):
    image_data = (np.squeeze(x_train_ran[id_n]) * 255).astype('uint8')
    image = Image.fromarray(image_data, 'L')
    
    title = f"Label:{y_train_random[id_n][0]}"
    print(title)
    display(image)

Label:3


Label:73


Label:40776


Label:1731


Label:8227


Label:343


Label:60


Label:09168


Label:8


Label:985


In [7]:
max_str_len = 5 # max length of input labels
num_of_characters = 11 # +1 for ctc pseudo blank
num_of_timestamps = 28 # max length of predicted labels

In [8]:
train_label_len = np.zeros([len(y_train_random), 1])
train_input_len = np.ones([len(y_train_random), 1]) * num_of_timestamps
train_output = np.zeros([len(y_train_random)])

for i in range(len(y_train_random)):
    train_label_len[i] = len(y_train_random[i][0]) 

print('True label : ',y_train_random[100] , '\ntrain_y : ',y_train_ran_ctc[100],'\ntrain_label_len : ',train_label_len [100], 
      '\ntrain_input_len : ', train_input_len[100])

True label :  ['51305'] 
train_y :  [5 1 3 0 5] 
train_label_len :  [5.] 
train_input_len :  [28.]


In [9]:
test_label_len = np.zeros([len(y_test_random), 1])
#test_input_len = np.ones([len(y_test_random), 1]) * (num_of_timestamps-2)
test_input_len = np.ones([len(y_test_random), 1]) * num_of_timestamps
test_output = np.zeros([len(y_test_random)])

for i in range(len(y_test_random)):
    test_label_len[i] = len(y_test_random[i][0]) 

print('True label : ',y_test_random[10] , '\ntrain_y : ',y_test_ran_ctc[10],'\ntrain_label_len : ',test_label_len [10], 
      '\ntrain_input_len : ', test_input_len[10])

True label :  ['450'] 
train_y :  [ 4  5  0 10 10] 
train_label_len :  [3.] 
train_input_len :  [28.]


In [10]:
verify_label_len = np.zeros([len(y_verify_random), 1])
verify_input_len = np.ones([len(y_verify_random), 1]) * num_of_timestamps
verify_output = np.zeros([len(y_verify_random)])

for i in range(len(y_verify_random)):
    verify_label_len[i] = len(y_verify_random[i][0]) 

print('True label : ',y_verify_random[1] , '\ntrain_y : ',y_verify_ran_ctc[1],'\ntrain_label_len : ',verify_label_len [1], 
      '\ntrain_input_len : ', verify_input_len[1])

True label :  ['6'] 
train_y :  [ 6 10 10 10 10] 
train_label_len :  [1.] 
train_input_len :  [28.]


In [11]:
import tensorflow as tf
from keras import backend as K
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Reshape, Bidirectional, LSTM, Dense, Lambda, Activation, BatchNormalization, Dropout
from keras.optimizers import Adam

input_data = Input(shape=(28, 100, 1), name='input')

inner = Conv2D(32, (3, 3), activation='relu', padding='same', name='conv1')(input_data)  
inner = BatchNormalization()(inner)
inner = Activation('relu')(inner)
inner = MaxPooling2D(pool_size=(2, 2), name='max1')(inner)

inner = Conv2D(64, (3, 3), activation='relu', padding='same', name='conv2')(inner)
inner = BatchNormalization()(inner)
inner = Activation('relu')(inner)
inner = MaxPooling2D(pool_size=(2, 2), name='max2')(inner)
inner = Dropout(0.3)(inner)



# CNN to RNN
inner = Reshape(target_shape=((28, 400)), name='reshape')(inner)
inner = Dense(64, activation="softmax", name='dense1')(inner)

## RNN
inner = Bidirectional(LSTM(128, return_sequences=True, dropout=0.25), name = 'lstm1')(inner)
inner = Bidirectional(LSTM(64, return_sequences=True, dropout=0.25), name = 'lstm2')(inner)

## OUTPUT
inner = Dense(num_of_characters,name='dense2')(inner)
y_pred = Activation('softmax', name='softmax')(inner)

model = Model(inputs=input_data, outputs=y_pred)
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 28, 100, 1)]      0         
_________________________________________________________________
conv1 (Conv2D)               (None, 28, 100, 32)       320       
_________________________________________________________________
batch_normalization (BatchNo (None, 28, 100, 32)       128       
_________________________________________________________________
activation (Activation)      (None, 28, 100, 32)       0         
_________________________________________________________________
max1 (MaxPooling2D)          (None, 14, 50, 32)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 14, 50, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 50, 64)       

In [12]:
# the ctc loss function
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    # the 2 is critical here since the first couple outputs of the RNN
    # tend to be garbage
    #y_pred = y_pred[:, 2:, :]
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

In [13]:

labels = Input(name='gtruth_labels', shape=[max_str_len], dtype='float32')
input_length = Input(name='input_length', shape=[1], dtype='int64')
label_length = Input(name='label_length', shape=[1], dtype='int64')

ctc_loss = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([y_pred, labels, input_length, label_length])
model_final = Model(inputs=[input_data, labels, input_length, label_length], outputs=ctc_loss)

In [14]:
# the loss calculation occurs elsewhere, so we use a dummy lambda function for the loss
from tensorflow import keras
from tensorflow.keras.callbacks import ReduceLROnPlateau 
model_final.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=keras.optimizers.Adam(learning_rate=0.01),
                  metrics=['accuracy'])


history = model_final.fit(x=[x_train_ran, y_train_ran_ctc, train_input_len, train_label_len], y=train_output, 
                validation_data=([x_test_ran, y_test_ran_ctc, test_input_len, test_label_len], test_output),
                epochs=100, verbose=1, batch_size=200)

Epoch 1/100
300/300 [==============================] - 214s 713ms/step - loss: 8.2956 - accuracy: 1.6667e-05 - val_loss: 7.0694 - val_accuracy: 0.0000e+00
Epoch 2/100
300/300 [==============================] - 224s 748ms/step - loss: 6.6306 - accuracy: 0.0192 - val_loss: 6.1939 - val_accuracy: 0.0349
Epoch 3/100
300/300 [==============================] - 208s 693ms/step - loss: 5.8519 - accuracy: 0.0498 - val_loss: 5.1140 - val_accuracy: 0.0812
Epoch 4/100
300/300 [==============================] - 213s 708ms/step - loss: 5.7389 - accuracy: 0.0632 - val_loss: 5.2145 - val_accuracy: 0.0892
Epoch 5/100
300/300 [==============================] - 203s 676ms/step - loss: 5.1086 - accuracy: 0.1233 - val_loss: 4.4891 - val_accuracy: 0.1557
Epoch 6/100
300/300 [==============================] - 199s 662ms/step - loss: 4.5617 - accuracy: 0.1480 - val_loss: 8.3029 - val_accuracy: 0.0046
Epoch 7/100
300/300 [==============================] - 198s 659ms/step - loss: 4.2326 - accuracy: 0.1658 - val

In [15]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2, subplot_titles=("Model accuracy", "Model loss"))

# Add the first subplot for accuracy
fig.add_trace(go.Scatter(x=list(range(1, len(history.history['accuracy']) + 1)),
                         y=history.history['accuracy'], mode='lines',
                         name='Train', line=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(x=list(range(1, len(history.history['val_accuracy']) + 1)),
                         y=history.history['val_accuracy'], mode='lines',
                         name='Validation', line=dict(color='orange')), row=1, col=1)

# Add the second subplot for loss
fig.add_trace(go.Scatter(x=list(range(1, len(history.history['loss']) + 1)),
                         y=history.history['loss'], mode='lines',
                         name='Train', line=dict(color='blue'), showlegend=False), row=1, col=2)
fig.add_trace(go.Scatter(x=list(range(1, len(history.history['val_loss']) + 1)),
                         y=history.history['val_loss'], mode='lines',
                         name='Validation', line=dict(color='orange'), showlegend=False), row=1, col=2)

fig.update_layout(title="Model Training History",
                  xaxis_title="Epoch",
                  yaxis_title="Value",
                  legend=dict(x=0.85, y=1))

fig.show()

In [16]:
preds = model.predict(x_verify_ran)
decoded = K.get_value(K.ctc_decode(preds, input_length=np.ones(preds.shape[0])*preds.shape[1], 
                                   greedy=True)[0][0])

alphabets = u"0123456789 "

def num_to_label(num):
    ret = ""
    for ch in num:
        if ch == -1:  # CTC Blank
            break
        else:
            ret+=alphabets[ch]
    return ret

prediction = []
for i in range(len(y_verify_ran)):
    prediction.append(num_to_label(decoded[i]))

Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.


In [35]:
y_true = []
for i in range(20000):
    y_true.append(y_verify_random[i][0])

In [36]:
correct_char = 0
total_char = 0
correct = 0
valid_size=len(y_verify_random)

for i in range(valid_size):
    pr = prediction[i]
    tr = y_true[i]
    total_char += len(tr)
    
    for j in range(min(len(tr), len(pr))):
        if tr[j] == pr[j]:
            correct_char += 1
            
    if pr == tr :
        correct += 1 
    
print('Correct characters predicted : %.2f%%' %(correct_char*100/total_char))
print('Correct words predicted      : %.2f%%' %(correct*100/valid_size))

Correct characters predicted : 94.66%
Correct words predicted      : 85.84%


In [38]:
for id_n in np.random.randint(0, len(y_true),10):
    image_data = (np.squeeze(x_verify_ran[id_n]) * 255).astype('uint8')
    image = Image.fromarray(image_data, 'L')
    title = f"Label:{y_true[id_n]}; Predicted:{prediction[id_n]}"
    print(title)
    display(image)

Label:118; Predicted:118


Label:0811; Predicted:0811


Label:9065; Predicted:9065


Label:80; Predicted:80


Label:4; Predicted:4


Label:49543; Predicted:49593


Label:533; Predicted:533


Label:151; Predicted:251


Label:1545; Predicted:1545


Label:88; Predicted:88
